In [2]:
import numpy as np
import random
import time
import copy
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor, as_completed
from threading import Thread
from multiprocessing import Pool
import multiprocessing

In [3]:
print(multiprocessing.cpu_count())

12


In [4]:
def tokens_2(input_string):
    if len(input_string) % 2 != 0:
        raise ValueError("Input string length must be even.")
    tokens = [input_string[i:i+2] for i in range(0, len(input_string), 2)]
    return tokens

In [5]:
class card:
    def __init__(self, rank, color, name):
        self.name = name
        self.rank = rank
        self.color = color
    def index(self):
        return (self.rank, self.color)


In [6]:
class map:
    def __init__(self):
        self.color_names = ['h', 's', 'd', 'c'] #columns
        self.rank_names = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A'] #rows
        self.card_map = {}
        for index_row in range(len(self.rank_names)):
            for index_col in range(len(self.color_names)):
                key = f"{self.rank_names[index_row]}{self.color_names[index_col]}"
                value = card(index_row, index_col, f"{self.rank_names[index_row]}{self.color_names[index_col]}")
                self.card_map[key] = value
        self.index_map = {}
        for index_row in range(len(self.rank_names)):
            for index_col in range(len(self.color_names)):
                key = (index_row, index_col)
                value = f"{self.rank_names[index_row]}{self.color_names[index_col]}"
                self.index_map[key] = value

    def card(self, name):
        return self.card_map[name]

    def card_index(self, name):
        return self.card_map[name].index()


In [7]:
card_map = map()
print(card_map.card_index('Ah'))
#print(card_map.index_map)
print(card_map.index_map[(12, 0)])

(12, 0)
Ah


In [8]:
class deck:
    card_map = map()
    def __init__(self, cards = '', blocked_cards = ''):
        #print(deck.card_map.card_map)
        self.card_map = map()
        self.card_array = np.zeros((13, 4))
        self.cards_names = tokens_2(cards)
        self.blocked_cards_names = tokens_2(blocked_cards)
        for name in self.cards_names:
            self.card_array[self.card_map.card_index(name)] = 1
        for name in self.blocked_cards_names:
            self.card_array[self.card_map.card_index(name)] = -1
    def deal_random(self):
        while True:
            random_rank = random.randint(0, 12)
            random_color = random.randint(0, 3)
            if self.card_array[random_rank, random_color] == 0:
                self.card_array[random_rank, random_color] = 1
                break

    def deal_card(self, name):
        if self.card_array[self.card_map.card_index(name)] == 1:
            self.card_array[self.card_map.card_index(name)] = 1

    def __add__(self, other):
        if isinstance(other, deck):
            tmp = copy.copy(self)
            tmp.card_array = tmp.card_array + other.card_array
            tmp.cards_names += other.cards_names
            tmp.blocked_cards_names += other.blocked_cards_names
            return tmp
        else:
            raise ValueError("'+' Operation not defined")

    def __sub__(self, other):
        if isinstance(other, deck):
            tmp = copy.copy(self)
            tmp.card_array = tmp.card_array - other.card_array
            for element in other.cards_names:
                if element in tmp.cards_names:
                    tmp.cards_names.remove(element)
            for element in other.blocked_cards_names:
                if element in tmp.blocked_cards_names:
                    tmp.blocked_cards_names.remove(element)
            return tmp
        else:
            raise ValueError("'-' Operation not defined")

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            tmp = copy.copy(self)
            tmp.card_array = tmp.card_array * 2
            return tmp
        else:
            raise ValueError("'*' Operation not defined")

In [9]:
class player_range: #input_string.split(',')
    def __init__(self, input_codes, blocked_cards = ''):
        self.color_names = ['h', 's', 'd', 'c'] #columns
        self.rank_names = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A'] #rows
        self.blocked_cards_names = tokens_2(blocked_cards)
        self.input_codes = input_codes.split(',')
        self.hands_list = []
        for input_code in self.input_codes:
            if len(input_code) == 3:
                if input_code[2] == 'o':
                    for color1 in self.color_names:
                        for color2 in self.color_names:
                            if color1 != color2:
                                card1 = f"{input_code[0]}{color1}"
                                card2 = f"{input_code[1]}{color2}"
                                if not (card1 in self.blocked_cards_names or card2 in self.blocked_cards_names):
                                    self.hands_list.append(f"{input_code[0]}{color1}{input_code[1]}{color2}")
                if input_code[2] == 's':
                    for color1, color2 in zip(self.color_names, self.color_names):
                        if color1 == color2:
                            card1 = f"{input_code[0]}{color1}"
                            card2 = f"{input_code[1]}{color2}"
                            if not (card1 in self.blocked_cards_names or card2 in self.blocked_cards_names):
                                self.hands_list.append(f"{input_code[0]}{color1}{input_code[1]}{color2}") 
            if len(input_code) == 2:                   
                if input_code[0] == input_code[1]:
                    card_name = input_code[0]
                    for index1 in range(len(self.color_names)):
                        for index2 in range(index1+1,len(self.color_names)):
                            if self.color_names[index1] != self.color_names[index2]:
                                card1 = f"{card_name}{self.color_names[index1]}"
                                card2 = f"{card_name}{self.color_names[index2]}"
                                if not (card1 in self.blocked_cards_names or card2 in self.blocked_cards_names):
                                    self.hands_list.append(f"{input_code[0]}{self.color_names[index1]}{input_code[1]}{self.color_names[index2]}")

In [10]:
def code_range(input_codes):
    rank_names = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A'] #rows
    input_codes = input_codes.split(',')
    output = ''
    for input_code in input_codes:
        if len(input_code) == 4 and input_code[3] == '+':
            high_name = input_code[0]
            low_name = input_code[1]
            high_index = rank_names.index(high_name)
            low_index = rank_names.index(low_name)
            for index in range(low_index, high_index):
                output = output + f"{rank_names[high_index]}{rank_names[index]}{input_code[2]},"
        if len(input_code) == 3 and input_code[2] == '+':
            if input_code[0] == input_code[1]:
                pair_name = input_code[0]
                pair_index = rank_names.index(pair_name)
                for index in range(pair_index, 13):
                    output = output + f"{rank_names[index]}{rank_names[index]},"
            if input_code[0] != input_code[1]:
                high_name = input_code[0]
                low_name = input_code[1]
                high_index = rank_names.index(high_name)
                low_index = rank_names.index(low_name)
                for index in range(low_index, high_index):
                    output = output + f"{rank_names[high_index]}{rank_names[index]}o,{rank_names[high_index]}{rank_names[index]}s,"
        if len(input_code) == 3 and input_code[2] != '+':
            output = output + input_code + ','
        if len(input_code) == 2:
            if input_code[0] == input_code[1]:
                output = output + f"{input_code[0]}{input_code[0]},"
            if input_code[0] != input_code[1]:
                output = output + f"{input_code[0]}{input_code[1]}s,{input_code[0]}{input_code[1]}o,"

    return output[0:-1]

In [11]:
#villian_range = player_range(input_codes="88", blocked_cards='8c8d8h')
villian_range = player_range(input_codes="AKo,88,99", blocked_cards='AhKd')
print(villian_range.hands_list)

['AsKh', 'AsKc', 'AdKh', 'AdKs', 'AdKc', 'AcKh', 'AcKs', '8h8s', '8h8d', '8h8c', '8s8d', '8s8c', '8d8c', '9h9s', '9h9d', '9h9c', '9s9d', '9s9c', '9d9c']


In [12]:
print((deck("","AhTd")-deck("","8c8d")*2).card_array)

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  2.  2.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [-1.  0.  0.  0.]]


In [13]:
cards = deck("2h3h4h5h6h")
array = cards.card_array
np.sum(array[0:0+5,:],axis = 0)

array([5., 0., 0., 0.])

In [14]:
print(np.vstack((array[0:0+4,:], array[12,:])))

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [15]:
def check_straights(cards):
    array = cards.card_array
    rank_sum = np.sum(array, axis=1) #asis 1 - suma po rankach #axis 0 suma po kolorach
    for n in range(-1, 10):
        is_straight = True
        for m in range(0, 5):
            if rank_sum[(n+m)%13] < 1:
                is_straight = False
                break
        if is_straight == True:
            if n != -1:
                if max(np.sum(array[n:n+5,:],axis = 0)) == 5:
                    return "straight flush", [n+4]
            elif max(np.sum(np.vstack((array[0:0+4,:], array[12,:])), axis=0)) == 5:
                return "straight flush", [3]
            else:
                return "straight", [n+4]
    return None, None

In [16]:
print(check_straights(deck("2h3h4h5hAh")))

('straight flush', [3])


In [17]:
def check_above_straight(cards):
    array = cards.card_array
    rank_sum = np.sum(array, axis=1) #asis 1 - suma po rankach #axis 0 suma po kolorach
    color_sum = np.sum(array, axis=0)
    if max(rank_sum) == 4:
        quad_index = np.where(rank_sum == 4)[0][0]
        rank_sum[quad_index] = 0
        return "quads", [quad_index, max(np.where(rank_sum > 0)[0])]
    elif np.size(np.where(rank_sum >= 3)[0]) >= 1 and np.size(np.where(rank_sum >= 2)[0]) >= 2:
        index_of_highest_trips = max(np.where(rank_sum >= 3)[0])
        rank_sum[index_of_highest_trips] = 0
        return "full house", [index_of_highest_trips, max(np.where(rank_sum >= 2)[0])]
    elif max(color_sum) >= 5:
        color_index = np.where(color_sum >= 5)[0]
        index_of_flush = np.where(array[:,color_index] == 1)[0]
        index_of_flush = np.sort(index_of_flush)[::-1]
        return "flush" , [index_of_flush[0], index_of_flush[1], index_of_flush[2], index_of_flush[3], index_of_flush[4]]
    else:
        return None, None


In [18]:
print(check_above_straight(deck("QcTc4cAc6cQsAs")))

('flush', [12, 10, 8, 4, 2])


In [19]:
def check_below_straight(cards):
    array = cards.card_array
    rank_sum = np.sum(array, axis=1) #asis 1 - suma po rankach #axis 0 suma po kolorach
    color_sum = np.sum(array, axis=0)
    if max(rank_sum) == 3:
        trips_index = max(np.where(rank_sum == 3)[0])
        high_index = np.where(rank_sum == 1)[0]
        high_index = np.sort(high_index)[::-1]
        return "trips", [trips_index, high_index[0], high_index[1]]
    elif np.size(np.where(rank_sum == 2)[0]) >= 2:
        pair_index = np.where(rank_sum == 2)[0]
        pair_index = np.sort(pair_index)[::-1]
        out1 = pair_index[0]
        out2 = pair_index[1]
        rank_sum[out1] = 0
        rank_sum[out2] = 0
        high_index = np.where(rank_sum >= 1)[0]
        return "two pairs", [out1, out2, max(high_index)]
    elif np.size(np.where(rank_sum == 2)[0]) == 1:
        pair_index = np.where(rank_sum == 2)[0][0]
        high_index = np.where(rank_sum == 1)[0]
        high_index = np.sort(high_index)[::-1]
        return "pair", [pair_index, high_index[0], high_index[1], high_index[2]]
    else:
        high_index = np.where(rank_sum == 1)[0]
        high_index = np.sort(high_index)[::-1]
        return "high card", [high_index[0], high_index[1], high_index[2], high_index[3], high_index[4]]



In [20]:
comb_rank = {"high card" : 0, "pair" : 1, "two pairs" : 2, "trips" : 3, "straight" : 4, "flush" : 5, "full house" : 6, "quads" : 7, "straight flush" : 8}

In [21]:
comb_rank["pair"] < comb_rank["two pairs"]

True

In [22]:
def check_combo(cards):
    comb, helpers = check_straights(cards)
    if comb == "straight flush":
        return comb, helpers
    if comb == "straight":
        comb2, helpers2 = check_above_straight(cards)
        if comb2 != None:
            return comb2, helpers2
        else:
            return comb, helpers
    if comb == None:
        comb2, helpers2 = check_above_straight(cards)
        if comb2 != None:
            return comb2, helpers2
        comb, helpers = check_below_straight(cards)
        return comb, helpers

In [23]:
def winner(combo1, helpers1, combo2, helpers2):
    if comb_rank[combo1] == comb_rank[combo2]:
        #print(helpers1, helpers2)
        for helper1, helper2 in zip(helpers1, helpers2):
            if helper1 > helper2:
                return 1
            if helper1 < helper2:
                return -1
        return 0
    if comb_rank[combo1] > comb_rank[combo2]:
        return 1
    if comb_rank[combo1] < comb_rank[combo2]:
        return -1

In [22]:
hero = deck("AhTd")
villian = deck("8c8d")
outcome_tab = np.array([0, 0, 0])
for n in range(10000):
    board = deck("","8c8dAhTd")
    for m in range(5):
        board.deal_random()
    hero_river = board + villian + hero*2
    villian_river = board + villian*2 + hero
    hero_combo, hero_kickers = check_combo(hero_river)
    villian_combo, villian_kickers = check_combo(villian_river)
    outcome = winner(hero_combo, hero_kickers, villian_combo, villian_kickers)
    outcome_tab[outcome+1] += 1

outcome_tab = outcome_tab / np.sum(outcome_tab)
print(outcome_tab)

[0.5566 0.0018 0.4416]


In [23]:
villian_range = player_range(input_codes=code_range('ATs+,AQo+,77+,65,74o,J2s'), blocked_cards='AhKd')
print(villian_range.hands_list)

['AsTs', 'AdTd', 'AcTc', 'AsJs', 'AdJd', 'AcJc', 'AsQs', 'AdQd', 'AcQc', 'AsKs', 'AcKc', 'AsQh', 'AsQd', 'AsQc', 'AdQh', 'AdQs', 'AdQc', 'AcQh', 'AcQs', 'AcQd', 'AsKh', 'AsKc', 'AdKh', 'AdKs', 'AdKc', 'AcKh', 'AcKs', '7h7s', '7h7d', '7h7c', '7s7d', '7s7c', '7d7c', '8h8s', '8h8d', '8h8c', '8s8d', '8s8c', '8d8c', '9h9s', '9h9d', '9h9c', '9s9d', '9s9c', '9d9c', 'ThTs', 'ThTd', 'ThTc', 'TsTd', 'TsTc', 'TdTc', 'JhJs', 'JhJd', 'JhJc', 'JsJd', 'JsJc', 'JdJc', 'QhQs', 'QhQd', 'QhQc', 'QsQd', 'QsQc', 'QdQc', 'KhKs', 'KhKc', 'KsKc', 'AsAd', 'AsAc', 'AdAc', '6h5h', '6s5s', '6d5d', '6c5c', '6h5s', '6h5d', '6h5c', '6s5h', '6s5d', '6s5c', '6d5h', '6d5s', '6d5c', '6c5h', '6c5s', '6c5d', '7h4s', '7h4d', '7h4c', '7s4h', '7s4d', '7s4c', '7d4h', '7d4s', '7d4c', '7c4h', '7c4s', '7c4d', 'Jh2h', 'Js2s', 'Jd2d', 'Jc2c']


In [24]:
def monte_carlo_hand(villian_hand, hero_hand = 'AhTd', n =10000):
    hero = deck(hero_hand)
    villian = deck(villian_hand)
    print(villian_hand)
    for m in range(n):
        board = deck("",(villian_hand+hero_hand))
        for l in range(5):
            board.deal_random()
        hero_river = board + villian + hero*2
        villian_river = board + villian*2 + hero
        hero_combo, hero_kickers = check_combo(hero_river)
        villian_combo, villian_kickers = check_combo(villian_river)
        outcome = winner(hero_combo, hero_kickers, villian_combo, villian_kickers)
    return outcome

In [25]:
def monte_carlo_list(villian_range):
    out = []
    for villian_hand in villian_range:
        out.append(monte_carlo_hand(villian_hand))
    return out

In [26]:
villian_range = player_range(input_codes=code_range('ATs+,AQo+,77+,65,74o,J2s'), blocked_cards='AhTd')
thread1 = Thread(target=monte_carlo_list, args=(villian_range.hands_list[0:50],))
thread2 = Thread(target=monte_carlo_list, args=(villian_range.hands_list[50:],))
thread1.start()
thread2.start()

thread1.join()
thread2.join()

AsTs
JhJs
JhJd
AcTc
JhJc
AsJs
JsJd
JsJc
AdJd
JdJc
AcJc
QhQs
AsQs
QhQd
AdQd
QhQc
AcQc
QsQd
AsKs
QsQc
AdKd
QdQc
AcKc
AsQh
KhKs
AsQd
KhKd
AsQc
KhKc
AdQh
KsKdAdQs

KsKc
KdKc
AdQc
AsAd
AcQh
AsAc
AcQs
AcQd
AdAc
6h5h
AsKh
6s5s
AsKd
AsKc
6d5d
AdKh
6c5c
AdKs
AdKc
6h5s
6h5d
AcKh
AcKs
6h5c
AcKd
6s5h
6s5d7h7s

7h7d
6s5c
7h7c
6d5h
6d5s
7s7d
6d5c
7s7c
6c5h
7d7c
6c5s
8h8s
6c5d
8h8d
7h4s
8h8c
7h4d
8s8d
7h4c
8s8c
8d8c
7s4h
9h9s
7s4d
9h9d
7s4c
9h9c
9s9d7d4h

9s9c
7d4s
9d9c
7d4c
ThTs
7c4h
ThTc
7c4sTsTc

7c4d
Jh2h
Js2s
Jd2d
Jc2c


In [27]:
num_threads = 10
hero = deck("AhTd")
outcome_tab = np.array([0, 0, 0])
villian_range = player_range(input_codes=code_range('ATs+,AQo+,77+,65,74o,J2s'), blocked_cards='AhTd')
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
     result = list(executor.map(monte_carlo_hand, villian_range.hands_list))

AsTs
AcTc
AsJs
AdJd
AcJc
AsQs
AdQd
AcQc
AsKs
AdKd
AcKc
AsQh
AsQd
AsQc
AdQh
AdQs
AdQc
AcQhAcQs

AcQd
AsKh
AsKd
AsKc
AdKh
AdKs
AdKc
AcKh
AcKs
AcKd
7h7s
7h7d
7h7c
7s7d
7s7c
7d7c
8h8s
8h8d
8h8c
8s8d
8s8c
8d8c
9h9s
9h9d
9h9c
9s9d
9s9c
9d9c
ThTs
ThTc
TsTc
JhJs
JhJd
JhJc
JsJd
JsJc
JdJc
QhQs
QhQd
QhQc
QsQd
QsQc
QdQc
KhKs
KhKd
KhKc
KsKd
KsKc
KdKc
AsAdAsAc

AdAc
6h5h
6s5s
6d5d
6c5c
6h5s
6h5d6h5c

6s5h
6s5d
6s5c
6d5h
6d5s
6d5c
6c5h
6c5s
6c5d
7h4s
7h4d
7h4c
7s4h
7s4d
7s4c
7d4h
7d4s
7d4c
7c4h
7c4s
7c4dJh2h

Js2sJd2d

Jc2c


In [28]:
# outcome_tab = np.array([0, 0, 0])
# villian_range = player_range(input_codes=code_range('ATs+,AQo+,77+,65,74o,J2s'), blocked_cards='AhTd')
# with Pool() as pool:
#     results = pool.map(monte_carlo_hand, villian_range.hands_list)

In [ ]:
hero = deck("AhTd")
outcome_tab = np.array([0, 0, 0])
villian_range = player_range(input_codes=code_range('ATs+,AQo+,77+,65,74o,J2s'), blocked_cards='AhTd')
for villian_hand in villian_range.hands_list:
    villian = deck(villian_hand)
    print(villian_hand)
    for n in range(10000):
        board = deck("",(villian_hand+"AhTd"))
        for m in range(5):
            board.deal_random()
        hero_river = board + villian + hero*2
        villian_river = board + villian*2 + hero
        hero_combo, hero_kickers = check_combo(hero_river)
        villian_combo, villian_kickers = check_combo(villian_river)
        outcome = winner(hero_combo, hero_kickers, villian_combo, villian_kickers)
        outcome_tab[outcome+1] += 1

outcome_tab = outcome_tab / np.sum(outcome_tab)
print(outcome_tab)

AsTs
AcTc
AsJs
AdJd
AcJc
AsQs
AdQd
AcQc
AsKs
AdKd
AcKc
AsQh
AsQd
AsQc
AdQh
AdQs
AdQc
AcQh
AcQs
AcQd
AsKh
AsKd
AsKc
AdKh
AdKs
AdKc
AcKh
AcKs
AcKd
7h7s
7h7d
7h7c
7s7d
7s7c
7d7c
8h8s
8h8d
8h8c
8s8d
8s8c
8d8c
9h9s
9h9d
9h9c
9s9d
9s9c
9d9c
ThTs
ThTc
TsTc
JhJs
JhJd
JhJc
JsJd
JsJc
JdJc
QhQs
QhQd
QhQc
QsQd
QsQc
QdQc
KhKs
KhKd
KhKc
KsKd
KsKc
KdKc
AsAd
AsAc
AdAc
6h5h
6s5s
6d5d
6c5c
6h5s
6h5d
6h5c
6s5h
6s5d
6s5c
6d5h
6d5s
6d5c
6c5h
6c5s
6c5d
7h4s
7h4d
7h4c
7s4h
7s4d
7s4c
7d4h
7d4s
7d4c
7c4h
7c4s
7c4d
Jh2h
Js2s
Jd2d
Jc2c
[0.56865049 0.03208738 0.39926214]


In [ ]:
print(check_below_straight(deck("QcThAdAc2c8d7h")))

('pair', [12, 10, 8, 6])


In [ ]:
comb1, kickers1 = check_combo(deck("2d3h4c5c6dQsAs"))
comb2, kickers2 = check_combo(deck("QdQh4cAc6dQsAs"))


print(winner(comb1, kickers1, comb2, kickers2))

-1


In [24]:
print(check_combo(deck("2d3h4c5c6dQsAs")))
print(check_combo(deck("QdQh4cAc6dQsAs")))
print(check_combo(deck("QcTc4cAc6cQsAs")))
print(check_combo(deck("QdQh4cAc6d6sAs")))
print(check_combo(deck("2c5c8hJhJcKcAc")))



('straight', [3])
('full house', [10, 12])
('flush', [12, 10, 8, 4, 2])
('two pairs', [12, 10, 4])
('flush', [12, 11, 9, 3, 0])


In [ ]:
cards = deck()
for n in range(7):
    cards.deal_random()
while check_combo(cards)[0] != "flush":
    cards = deck()
    for n in range(7):
        cards.deal_random()
print(check_combo(cards))
print(cards.card_array)

('flush', [12, 11, 3, 1, 0])
[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


In [ ]:
cards = deck()
for n in range(7):
    cards.deal_random()

start_time = time.time()
print(check_combo(cards))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"check: {elapsed_time} seconds")

print(cards.card_array)

('pair', [1, 9, 8, 4])
check: 0.0005002021789550781 seconds
[[0. 1. 0. 0.]
 [1. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
print(cards.card_array)

[[0. 1. 0. 0.]
 [1. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
cards = deck("AhTd","8c9c")
print(cards.card_array)
cards.deal_random()
cards.deal_random()
print(cards.card_array)


[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 1.  0.  0.  0.]]
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 1.  0.  1.  0.]]


In [ ]:
hand_array = np.zeros((13, 4))

print(hand_array)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
hand_array[card_map['Td'].rank,card_map['Td'].color] += 1
print(hand_array)

TypeError: 'map' object is not subscriptable

In [ ]:
x = card_map['Td'].index()
hand_array[x] += 1
print(hand_array)


In [ ]:
# def make_map():
#     columns = ['h', 's', 'd', 'c']
#     rows = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']
#     card_map = {}
#     for index_row in range(len(rows)):
#         for index_col in range(len(columns)):
#             key = f"{rows[index_row]}{columns[index_col]}"
#             value = card(12-index_row, index_col, f"{rows[index_row]}{columns[index_col]}")
#             card_map[key] = value
#     return card_map

In [ ]:
# columns = ['h', 's', 'd', 'c']
# rows = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']

# card_map = {}

# for index_row in range(len(rows)):
#     for index_col in range(len(columns)):
#         key = f"{rows[index_row]}{columns[index_col]}"
#         value = card(12-index_row, index_col, f"{rows[index_row]}{columns[index_col]}")
#         card_map[key] = value

# for key, value in card_map.items():
#     print(f"{key}: {value.name}")

# value_at_Ah = card_map['Ah']
# print(f"The value at A and h is: {value_at_Ah}")